# Universidad del Valle de Guatemala
18 Avenida 11-95 Guatemala      
Departamento de Computación     
Facultad de Ingeniería       
Minería de Datos Sección 10         
Fecha de entrega: 29/01/2024        

Grupo 6 | Integrantes:
- Diego Alexander Hernández Silvestre 21270
- Linda Inés Jimenez Vides 21169
- Daniel Armando Valdez Reyes 21240

## Hoja  de trabajo 2
Para realizar una exploración inicial del conjunto de datos, se prodeció a realizar un resumen de la información proporcionada en el dataset del archivo "*movies.csv*". A continuación se muestran los resultados: 

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import seaborn as sb
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import sklearn.preprocessing
import scipy.cluster.hierarchy as sch
import skfuzzy as fuzz
import pylab
import sklearn.mixture as mixture
import pyclustertend 
import random
from typing import *
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

In [21]:


dataframe: pd.DataFrame = pd.read_csv("movies.csv",  encoding='ISO-8859-1')
rows_to_correct = dataframe[dataframe['actorsCharacter'].astype(str) == dataframe['id'].astype(str)]

# Realiza la corrección de valores
dataframe.loc[rows_to_correct.index, 'director'] = ''
dataframe.loc[rows_to_correct.index, 'actors'] = rows_to_correct['director']
dataframe.loc[rows_to_correct.index, 'actorsPopularity'] = rows_to_correct['actors']
dataframe.loc[rows_to_correct.index, 'actorsCharacter'] = rows_to_correct['actorsPopularity']

rows_to_correct = dataframe[dataframe['actorsPopularity'].astype(str) == 'TRUE']
dataframe.loc[rows_to_correct.index, 'actorsPopularity'] = rows_to_correct['video']
dataframe.loc[rows_to_correct.index, 'video'] = rows_to_correct['actors']

rows_to_correct = dataframe[dataframe['actorsPopularity'].astype(str) == 'FALSE']
dataframe.loc[rows_to_correct.index, 'actorsPopularity'] = rows_to_correct['video']
dataframe.loc[rows_to_correct.index, 'video'] = rows_to_correct['actors']


dataframe['castWomenAmount'] = pd.to_numeric(dataframe['castWomenAmount'], errors='coerce')
dataframe['castMenAmount'] = pd.to_numeric(dataframe['castMenAmount'], errors='coerce')

dataframeTo = dataframe.drop('id', axis=1)

dataframeTo.describe()

,budget,revenue,runtime,popularity,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
count,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9963.000000,9838.000000
mean,1.855163e+07,5.673793e+07,100.268100,51.393907,6.483490,1342.381800,2.596500,3.171400,1.751000,2147.666600,7004.157784,16557.193434
std,3.662669e+07,1.495854e+08,27.777829,216.729552,0.984274,2564.196637,1.154565,2.539738,3.012093,37200.075802,67239.619658,100455.960117
min,0.000000e+00,0.000000e+00,0.000000,4.258000,1.300000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,90.000000,14.577750,5.900000,120.000000,2.000000,2.000000,1.000000,13.000000,3.000000,7.000000
50%,5.000000e+05,1.631245e+05,100.000000,21.905500,6.500000,415.000000,3.000000,3.000000,1.000000,21.000000,6.000000,12.000000
75%,2.000000e+07,4.479661e+07,113.000000,40.654000,7.200000,1316.000000,3.000000,4.000000,2.000000,36.000000,10.000000,20.000000
max,3.800000e+08,2.847246e+09,750.000000,11474.647000,10.000000,30788.000000,16.000000,89.000000,155.000000,919590.000000,922162.000000,922017.000000


In [22]:
import numpy as np
# Dividir los valores y convertir a listas
dataframe = dataframe.assign(actorsPopularityStrArray=dataframe['actorsPopularity'].str.split('|', expand=False))

def convert_to_numeric(lst):
    if lst:
        if isinstance(lst, list):
            return [float(value) for value in lst]
        else:
            return [float(lst)]
    else:
        return []


dataframe = dataframe.assign(actorsPopularityNumArray=dataframe['actorsPopularityStrArray'].apply(lambda x: convert_to_numeric(x) if x is not None else x))

dataframe = dataframe.assign(actorsMostPopularity= dataframe['actorsPopularityNumArray'].apply(lambda x: max(x)))
dataframe = dataframe.assign(actorsWorstPopularity= dataframe['actorsPopularityNumArray'].apply(lambda x: min(x)))
dataframe = dataframe.assign(actorsPopularityMean= dataframe['actorsPopularityNumArray'].apply(lambda x: np.mean(x)))
dataframe = dataframe.assign(actorsPopularityMedian= dataframe['actorsPopularityNumArray'].apply(lambda x: np.median(x)))

dataframe = dataframe[['popularity', 'budget', 'revenue', 'runtime', 'genresAmount',
                    'productionCoAmount', 'productionCountriesAmount', 'actorsAmount', 'castWomenAmount', 'castMenAmount', 
                    'actorsMostPopularity', 
                    'actorsWorstPopularity', 
                    'actorsPopularityMean',
                    'actorsPopularityMedian']]
dataframe

,popularity,budget,revenue,runtime,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount,actorsMostPopularity,actorsWorstPopularity,actorsPopularityMean,actorsPopularityMedian
0,20.880,4000000,4257354.0,98,2,2,1,25,15.0,9.0,27.973,0.6,11.580640,7.9060
1,9.596,21000000,12136938.0,110,3,3,2,15,3.0,9.0,18.295,0.6,6.476200,4.3690
2,100.003,11000000,775398007.0,121,3,2,1,105,5.0,62.0,24.542,0.6,2.135924,0.9910
3,134.435,94000000,940335536.0,100,2,1,1,24,5.0,18.0,33.379,0.6,8.139208,8.3395
4,58.751,55000000,677387716.0,142,3,2,1,76,18.0,48.0,33.297,0.6,3.654921,1.6115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16.662,0,0.0,100,2,1,1,10,2.0,4.0,18.185,0.6,2.678400,0.6000
9996,491.706,0,0.0,97,1,2,1,8,1.0,1.0,1.550,0.6,0.748750,0.6000
9997,565.658,0,0.0,112,1,1,17,1,0.0,922017.0,1.400,0.6,0.730706,0.6000
9998,9.664,0,0.0,59,1,0,0,0,922162.0,NaN,NaN,NaN,NaN,NaN


Descripción de las variables

- Id: Cualitativa Ordinal
- popularity: Cuantitativa Discreta
- budget: Cuantitativa Discreta
- revenue: Cuantitativa Discreta
- original_title: Cualitativa Nominal
- originalLanguage: Cualitativa Nominal
- title: Cualitativa Nominal
- homePage: Cualitativa Nominal
- video: Cualitativa Nominal
- director: Cualitativa Nominal
- runtime: Cuantitativa Continua
- genres: Cualitativa Nominal
- genresAmount: Cuantitativa Discreta
- productionCompany: Cualitativa Nominal
- productionCoAmount: Cuantitativa Discreta
- productionCompanyCountry: Cualitativa Nominal
- productionCountry: Cualitativa Nominal
- productionCountriesAmount: Cuantitativa Contable
- releaseDate: Cualitativa Ordinal
- voteCount: Cuantitativa Contable
- voteAvg: Cuantitativo Contable
- actors: Cualitativo Nominal
- actorsPopularity: Cuantitativa Discreta
- actorsCharacter: Cualitativa Nominal
- actorsAmount: Cuantitativa Discreta
- castWomenAmount: Cuantitativa Discreta
- castMenAmount: Cuantitativa Discreta

Descripción de variables a utilizar:
- popularity: Cuantitativa Discreta
- budget: Cuantitativa Discreta
- revenue: Cuantitativa Discreta
- runtime: Cuantitativa Continua
- genresAmount: Cuantitativa Discreta
- productionCoAmount: Cuantitativa Discreta
- productionCountriesAmount: Cuantitativa Contable
- actorsPopularity: Cuantitativa Discreta
- actorsAmount: Cuantitativa Discreta
- castWomenAmount: Cuantitativa Discreta
- castMenAmount: Cuantitativa Discreta

In [23]:
selected_columns = dataframe[['popularity', 'budget', 'revenue', 'runtime', 'genresAmount',
                              'productionCoAmount', 'productionCountriesAmount', 
                              'actorsAmount', 'castWomenAmount', 'castMenAmount']]

# Agregar la columna 'actorsPopularity' con los valores numéricos
selected_columns = selected_columns.assign(actorsPopularity=split_values_numeric)

# Imprimir el DataFrame resultante
print(selected_columns)




NameError: name 'split_values_numeric' is not defined